# Kubernetes RBAC — Role-Based Access Control

RBAC controls who can perform what actions on which resources in a Kubernetes cluster. It's essential for multi-tenant clusters, security compliance, and the principle of least privilege.

## Core concepts

| Resource | Scope | Description |
| --- | --- | --- |
| Role | Namespace | Defines permissions within a namespace |
| ClusterRole | Cluster-wide | Defines permissions across the cluster |
| RoleBinding | Namespace | Grants Role to users/groups/ServiceAccounts |
| ClusterRoleBinding | Cluster-wide | Grants ClusterRole cluster-wide |
| ServiceAccount | Namespace | Identity for pods to authenticate to API |

## RBAC model

```
Subject (Who?)          Role/ClusterRole (What permissions?)
    │                              │
    │    ┌─────────────────────────┘
    │    │
    ▼    ▼
RoleBinding / ClusterRoleBinding
           │
           ▼
    Resources (pods, services, secrets...)
```

**Subjects can be:**
- Users (external identity)
- Groups (external identity group)
- ServiceAccounts (in-cluster identity)

## Role and ClusterRole

```yaml
# Namespace-scoped Role
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  namespace: development
  name: pod-reader
rules:
- apiGroups: [""]           # "" = core API group
  resources: ["pods"]
  verbs: ["get", "watch", "list"]
- apiGroups: [""]  
  resources: ["pods/log"]
  verbs: ["get"]

---
# Cluster-wide ClusterRole
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: secret-reader
rules:
- apiGroups: [""]  
  resources: ["secrets"]
  verbs: ["get", "watch", "list"]
```

**Common verbs:** `get`, `list`, `watch`, `create`, `update`, `patch`, `delete`, `deletecollection`

## ServiceAccount

```yaml
# Create a ServiceAccount
apiVersion: v1
kind: ServiceAccount
metadata:
  name: my-app
  namespace: production

---
# Use in a Pod
apiVersion: v1
kind: Pod
metadata:
  name: my-pod
spec:
  serviceAccountName: my-app
  containers:
  - name: app
    image: myapp:latest
```

Every namespace has a `default` ServiceAccount. Pods use it unless specified otherwise.

## RoleBinding and ClusterRoleBinding

```yaml
# Bind Role to ServiceAccount in same namespace
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: read-pods
  namespace: development
subjects:
- kind: ServiceAccount
  name: my-app
  namespace: development
roleRef:
  kind: Role
  name: pod-reader
  apiGroup: rbac.authorization.k8s.io

---
# Bind ClusterRole to user across cluster
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: admin-binding
subjects:
- kind: User
  name: jane@example.com
  apiGroup: rbac.authorization.k8s.io
- kind: Group
  name: platform-team
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: ClusterRole
  name: cluster-admin
  apiGroup: rbac.authorization.k8s.io
```

## Built-in ClusterRoles

| ClusterRole | Permissions |
| --- | --- |
| `cluster-admin` | Full access to everything |
| `admin` | Full access within a namespace |
| `edit` | Read/write most resources, no RBAC |
| `view` | Read-only access to most resources |

## kubectl RBAC commands

```bash
# Check if you can perform an action
kubectl auth can-i create pods
kubectl auth can-i delete secrets --namespace production

# Check as another user/ServiceAccount
kubectl auth can-i list pods --as=system:serviceaccount:default:my-app
kubectl auth can-i '*' '*' --as=jane@example.com

# List all permissions for current user
kubectl auth can-i --list

# View RBAC resources
kubectl get roles,rolebindings -n development
kubectl get clusterroles,clusterrolebindings

# Describe a role
kubectl describe role pod-reader -n development

# Create role imperatively
kubectl create role pod-reader \
  --verb=get,list,watch \
  --resource=pods \
  -n development

# Create rolebinding imperatively  
kubectl create rolebinding read-pods \
  --role=pod-reader \
  --serviceaccount=development:my-app \
  -n development
```

## Mini example: RBAC permission matrix

In [ ]:
import pandas as pd
import plotly.express as px

# Common RBAC patterns
permissions = pd.DataFrame({
    "Role": ["cluster-admin", "admin", "edit", "view", "pod-reader"],
    "Pods": ["full", "full", "full", "read", "read"],
    "Secrets": ["full", "full", "full", "none", "none"],
    "ConfigMaps": ["full", "full", "full", "read", "none"],
    "RBAC": ["full", "none", "none", "none", "none"],
    "Nodes": ["full", "none", "none", "none", "none"],
})

# Convert to heatmap format
perm_map = {"full": 3, "read": 2, "none": 0}
perm_numeric = permissions.set_index("Role").replace(perm_map)

fig = px.imshow(
    perm_numeric,
    labels=dict(x="Resource", y="Role", color="Access Level"),
    color_continuous_scale=["white", "lightblue", "blue", "darkblue"],
    title="RBAC permission levels by role"
)
fig.update_layout(coloraxis_showscale=False)
fig.show()

print("Legend: 0=none, 2=read, 3=full")

## Best practices

1. **Least privilege** — Grant minimum permissions needed
2. **Use namespaces** — Isolate teams/apps with namespace-scoped roles
3. **Prefer Roles over ClusterRoles** — Limit cluster-wide access
4. **Avoid wildcards** — Don't use `*` for resources or verbs
5. **Audit regularly** — Review bindings with `kubectl get rolebindings -A`
6. **Use groups** — Bind to groups rather than individual users
7. **Don't modify default ServiceAccount** — Create specific ones per app